Import the db_utils module containing the write_delta_v2 function

In [12]:
df = spark.sql("SELECT * FROM AWS_LAKEHOUSE.SALESDETAILS LIMIT 1000")
display(df)

StatementMeta(, 7610eb0d-6566-4954-b635-eda921876c2d, 18, Finished, Available)

SynapseWidget(Synapse.DataFrame, aa42fbbe-ef50-4371-9382-bd09702df032)

In [13]:
df = spark.sql("SELECT * FROM AWS_LAKEHOUSE.SALESORDERS LIMIT 1000")
display(df)

StatementMeta(, 7610eb0d-6566-4954-b635-eda921876c2d, 19, Finished, Available)

SynapseWidget(Synapse.DataFrame, b1c8b915-b9ba-47d9-b40d-167a91e093f8)

In [14]:
%run /db_utils

StatementMeta(, , -1, Finished, Available)

In [1]:
# Define database connection properties for a PostgreSQL database
database_host = "intechpostgre.ckvdytywefte.eu-north-1.rds.amazonaws.com"
database_port = '5432'
database_name = 'intechdb'
user = "admin123"
password = "Intech1234"
driver = "org.postgresql.Driver"

# Define the JDBC URL to connect to the PostgreSQL database
url = f"jdbc:postgresql://{database_host}:{database_port}/{database_name}"

StatementMeta(, d22ed521-85c8-43c6-8a85-7ee94e2ca7f0, 5, Finished, Available)

In [2]:
# Query to get all the table names inside the 'public' schema of the database
query = f"(SELECT tablename FROM pg_catalog.pg_tables where schemaname = 'public' )  as tempTable"

# Read table names from the PostgreSQL database
table_names = (spark.read
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("dbtable", query)
    .option("user", user)
    .option("password", password)
    .load()
    )

# Print the number of table names retrieved
print(table_names.count())

# Show the retrieved table names
table_names.show()

# Collect the table names into a Python list
table_names_list = table_names.rdd.map(lambda row: row[0]).collect()

StatementMeta(, d22ed521-85c8-43c6-8a85-7ee94e2ca7f0, 6, Finished, Available)

Py4JJavaError: An error occurred while calling o4736.load.
: org.postgresql.util.PSQLException: The connection attempt failed.
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:292)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:49)
	at org.postgresql.jdbc.PgConnection.<init>(PgConnection.java:211)
	at org.postgresql.Driver.makeConnection(Driver.java:458)
	at org.postgresql.Driver.connect(Driver.java:260)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1(JdbcDialects.scala:122)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1$adapted(JdbcDialects.scala:118)
	at org.apache.spark.sql.execution.datasource

In [17]:
# Define the action to be performed on Delta tables ('upsert' or 'overwrite')
# action = 'overwrite'
action = 'upsert'

StatementMeta(, 7610eb0d-6566-4954-b635-eda921876c2d, 24, Finished, Available)

In [18]:
# Define a function to replace spaces with underscores in column names
def replace_space_with_underscore(column_name):
    return col(column_name).alias(column_name.replace(" ", "_"))

# Define a function to replace dashes with underscores in column names
def replace_dash_with_underscore(column_name):
    return col(column_name).alias(column_name.replace("-", "_"))

StatementMeta(, 7610eb0d-6566-4954-b635-eda921876c2d, 25, Finished, Available)

In [19]:
# Iterate through the retrieved table names
for table_name in table_names_list:
    
    tgt_table_name = table_name

    # Construct a SQL query to select all data from the current table
    query = f"(SELECT * FROM {table_name}) as tempTable"
    
    # Read data from the PostgreSQL table into a Spark DataFrame
    df = (spark.read
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("dbtable", query)
    .option("user", user)
    .option("password", password)
    .load()
    )

    # Replace spaces with underscores in column names
    df = df.select([replace_space_with_underscore(col_name) for col_name in df.columns])
    
    # Replace dashes with underscores in column names
    df = df.select([replace_dash_with_underscore(col_name) for col_name in df.columns])

    # Define the target database name
    tgt_db_name = 'tgt_db_name'

    # Convert the table name to uppercase (Delta table naming convention)
    table_name = table_name.upper()
    tgt_table_name = table_name
    
    # Write the DataFrame into a Delta Lake table
    if action == 'upsert':
        write_delta_v2(type='data_copy', df=df, mode=action, dbname=tgt_db_name, targetTable=tgt_table_name, sourceTable=tgt_table_name, keyCol='Order_ID')
    else:
        write_delta_v2(type='data_copy', df=df, mode=action, dbname=tgt_db_name, targetTable=tgt_table_name)

StatementMeta(, 7610eb0d-6566-4954-b635-eda921876c2d, 26, Finished, Available)

table already exists!
upsert mode
table already exists!
upsert mode


In [20]:
df = spark.sql("SELECT * FROM AWS_LAKEHOUSE.SALESDETAILS LIMIT 1000")
display(df)

StatementMeta(, 7610eb0d-6566-4954-b635-eda921876c2d, 27, Finished, Available)

SynapseWidget(Synapse.DataFrame, a53439a0-f7af-422d-8f39-4a6b7362cc58)

In [21]:
df = spark.sql("SELECT * FROM AWS_LAKEHOUSE.SALESORDERS LIMIT 1000")
display(df)

StatementMeta(, 7610eb0d-6566-4954-b635-eda921876c2d, 28, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3405124c-9508-4e18-bcb4-ffcbf49e9764)